In [1]:
import netgen.gui
from ngsolve import *
import scipy.sparse as sp
from ngsolve import internal
from netgen.geom2d import *
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

In [ ]:
def AllEw(A,B):
    Binv = np.linalg.inv(B)
    lam,v = np.linalg.eig(Binv.dot(A))
    return lam

In [ ]:
geo = SplineGeometry()
geo.AddRectangle ( (0,0), (1,1), bcs=["bottom","right","top","left"], leftdomain=1)

n = 3
r = 0.005
geo.AddRectangle((0.45,0.45),(0.5, 0.55), bc="interface",leftdomain=0, rightdomain=1)
geo.AddRectangle((0.45,0.44),(0.5, 0.001), bc="interface",leftdomain=0, rightdomain=1)
geo.AddRectangle((0.45,0.56),(0.5, 0.999), bc="interface",leftdomain=0, rightdomain=1)
#geo.AddCircle ( (0.5,0.48), r=r, bc="interface",leftdomain=0, rightdomain=1)
#geo.AddCircle ( (0.5,0.52), r=r, bc="interface",leftdomain=0, rightdomain=1)
#geo.AddCircle ( (0.5,0.5), r=r, bc="interface",leftdomain=0, rightdomain=1)
#geo.AddCircle ( (0.5,(1)/(3)), r=r, bc="interface",leftdomain=0, rightdomain=1)
                     
geo.SetMaterial(1, "medium")
# geo.SetMaterial(2, "dot")

mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw (mesh)

In [10]:
geo = SplineGeometry()
geo.AddRectangle ( (0,0), (1,1), bcs=["bottom","right","top","left"], leftdomain=1)

n = 40
r = 0.005
for i in range(n-1):
    geo.AddCircle ( (0.5,(i+1)/(n)), r=r, bc="interface", 
                     leftdomain=0, rightdomain=1)
geo.SetMaterial(1, "medium")


mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw (mesh)

In [11]:
source = exp(-10**2*(y-0.5)*(y-0.5))
Draw(source,mesh,name="sorce")

In [16]:
pi = 3.141425
maxind = 20
omegfact = np.linspace(40,60,maxind)
fes = H1(mesh, order=5, complex=True, dirichlet="interface")
gfu = GridFunction(fes)
erg = np.zeros([len(gfu.vec),maxind],dtype=complex)
n = { "medium" : 1, "dot" : 3 }
cfn = CoefficientFunction( [n[mat] for mat in mesh.GetMaterials()] )


u = fes.TrialFunction()
v = fes.TestFunction()

A = BilinearForm(fes)
R = BilinearForm(fes)
K = BilinearForm(fes)
M = BilinearForm(fes)
F = LinearForm(fes)

omega = Parameter(1)
A += (grad(u)*grad(v)- omega**2*cfn*cfn*u*v)*dx
A += -1j*omega*u*v*ds("left|right|top|bottom")
K += grad(u)*grad(v)*dx
R += -1j*u*v*ds("left|right|top|bottom")
M += cfn*cfn*u*v*dx

F = LinearForm(fes)

F += source*v*ds("left")

o = []
norm = []
for i in range(maxind):
    o.append(2*pi*omegfact[i])
#o = [2*pi*50]
j = 0
for i in o:   
    with TaskManager():
        omega.Set(i) 
        A.Assemble()
        K.Assemble()
        M.Assemble()
        F.Assemble()
        R.Assemble()   
        inv = A.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
        gfu.vec.data = inv * F.vec
        erg[:,j] = gfu.vec
        fh = F.vec
        norm.append(sqrt (Integrate ((Conj(gfu))*(gfu),mesh)))
        print("Number of iter: ",j,"omega: ",i,"L2-Norm:",sqrt (Integrate ((Conj(gfu))*(gfu),mesh)))
        j = j+1
    
Draw (gfu,mesh,name = "u_h")


rows,cols,vals = M.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = K.mat.COO()
Kh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = R.mat.COO()
Rh = sp.csr_matrix((vals,(rows,cols)))


Number of iter:  0 omega:  251.314 L2-Norm: (0.0007119328490967492+7.355958213524856e-24j)
Number of iter:  1 omega:  257.92752631578946 L2-Norm: (0.0006930865559650687+7.140635699807646e-24j)
Number of iter:  2 omega:  264.54105263157896 L2-Norm: (0.0006757271174874931+6.807051390114775e-24j)
Number of iter:  3 omega:  271.1545789473684 L2-Norm: (0.0006600990497361615-1.9611924976151344e-23j)
Number of iter:  4 omega:  277.7681052631579 L2-Norm: (0.0006458688491090847+2.233827266356696e-23j)
Number of iter:  5 omega:  284.38163157894735 L2-Norm: (0.0006324730045148813-6.76235602519117e-23j)
Number of iter:  6 omega:  290.99515789473685 L2-Norm: (0.0006198999713513087+5.802480803532988e-23j)
Number of iter:  7 omega:  297.6086842105263 L2-Norm: (0.0006075668549537969+1.8429429015185903e-23j)
Number of iter:  8 omega:  304.2222105263158 L2-Norm: (0.0005959690235061452-3.5313432748762196e-23j)
Number of iter:  9 omega:  310.83573684210523 L2-Norm: (0.0005847655137329321-3.647173991962622

In [13]:
V, r = np.linalg.qr(erg)
V = V.dot(np.linalg.inv(r))

In [14]:
MN = np.transpose(V).dot(Mh.dot(V))
KN = np.transpose(V).dot(Kh.dot(V))
RN = np.transpose(V).dot(Rh.dot(V))
fN = np.transpose(V).dot(fh)

In [22]:
ured = GridFunction(fes)
norm_s =[]
Residual_s=[]
#ngs_temp = K.mat.CreateColVector()
#ngs_temp2 = K.mat.CreateColVector()
uN_s = np.zeros([maxind,maxind])
j = 0
for fre in o:
    ngs_temp = K.mat.CreateColVector()
    ngs_temp2 = K.mat.CreateColVector()
    uN_s[:,j] = np.linalg.solve(KN-fre**2*MN+fre*RN, fN)
    print("Omega: ",fre)
    norm_s.append(np.linalg.norm(uN_s[:,j]))

    ured.vec.FV().NumPy()[:] = uN_s[:,j].dot(np.transpose(V))
    with TaskManager():
        ngs_temp.data = K.mat*ured.vec - fre*fre*M.mat*ured.vec + fre * R.mat*ured.vec - F.vec
        ngs_temp2.data = Projector(fes.FreeDofs(), True)*ngs_temp                                      
        res = Norm(ngs_temp2)
        Residual_s.append(res)
    j = j + 1
Residual_s    

Omega:  251.314


/home/alexander/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()


IndexError: boolean index did not match indexed array along dimension 0; dimension is 20 but corresponding boolean dimension is 155232

In [20]:
[fes.FreeDofs()]

In [9]:
u_N_s = GridFunction(fes)
u_N_s.vec.FV().NumPy()[:] = uN_s[:,19].dot(np.transpose(V))
Draw(u_N_s,mesh,name="u_N_s")
Draw(u_N_s-gfu,mesh,name="u_err")

In [ ]:
ured = GridFunction(fes)
iteration = int(1000)
Start = o[0] 
Ende = o[maxind-1]
lam = []
factor = (Ende-Start) / iteration
omegaN = []
norm_L2 =[]
Residual=[]
ngs_temp = K.mat.CreateColVector()
ngs_temp2 = K.mat.CreateColVector()
uN = np.zeros([maxind,iteration])
for i in range(iteration):
    omegaN.append(Start+factor*i)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN+omegaN[i]*RN, fN)
    print("i: ",i,"Omega: ",omegaN[i])
    norm_L2.append(sqrt(np.vdot(uN[:,i],uN[:,i])))

    ured.vec.FV().NumPy()[:] = uN[:,i].dot(np.transpose(V))
    with TaskManager():
        ngs_temp.data = K.mat*ured.vec - omegaN[i]*omegaN[i]*M.mat*ured.vec + omegaN[i] * R.mat*ured.vec - F.vec
        ngs_temp2.data = Projector(fes.FreeDofs(), True)*ngs_temp                                      
        res = Norm(ngs_temp2)
        Residual.append(res)

In [ ]:
print(Residual[53],o[1])

In [ ]:
l=AllEw(KN,MN+RN)
l = np.sqrt(l)

In [ ]:
def DrawOmega(amp):
    gfu_N = GridFunction(fes)
    gfu_N.vec.FV().NumPy()[:] = uN[:,amp].dot(np.transpose(V))
    print(omegaN[amp])
    Draw (gfu_N,mesh,name="u_N")
    internal.visoptions.autoscale = False
    

In [ ]:
#x = np.linspace(0,10,1000)
amp = widgets.IntSlider(min=0, max=iteration-1, value=4, description = "Omega:")
widgets.interactive(DrawOmega,amp=amp)

In [ ]:
d = []
for i in range(20):
    d.append(0.1)

In [ ]:
plt.plot(omegaN,Residual)
#plt.plot(o,d,"ro")
plt.plot(np.real(l[2:]),d[2:],"bo")

In [ ]:
plt.plot(omegaN,norm_L2)
plt.plot(np.real(l[2:]),d[2:],"bo")